In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
cd /content/gdrive/MyDrive/content_2

In [ ]:
import os
import pathlib

In [ ]:
# Clone the tensorflow model repository
if "models" in pathlib.Path.cwd().parts:
 while "models" in pathlib.Path.cwd().parts:
   os.chdir('..')
elif not pathlib.Path('models').exists():
 !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install API
%%bash
cd /content/gdrive/MyDrive/content_2/models/research
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
 
import tensorflow as tf
 
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder
 
%matplotlib inline

In [ ]:
#run model test
!python /content/gdrive/MyDrive/content_2/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
#Save train and test TFrecords to directory
%cd /content/gdrive/MyDrive/content_2
!curl -L "https://app.roboflow.com/ds/N0syuxJPXg?key=jIqlzNOpbK" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
#Select this for ssd mobilenet 
MODELS_CONFIG = {
   'ssd_mobilenet_640': {
       'model_name': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8',
       'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config',
       'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz',
       'batch_size': 8
   }
}

In [ ]:
##Select this for  ssd resnet
MODELS_CONFIG = {
   'ssd_resnet_640': {
       'model_name': 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8',
       'base_pipeline_file': 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config',
       'pretrained_checkpoint': 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz',
       'batch_size': 8
   }
}

In [ ]:
#Select the required model and specify training parameter
chosen_model = 'ssd_mobilenet_640'
num_steps = 50000 
num_eval_steps = 500
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] 

In [ ]:
#download pretrained weights
%mkdir /content/gdrive/MyDrive/content_2/models/research/deploy/
%cd /content/gdrive/MyDrive/content_2/models/research/deploy/
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

In [ ]:
#download base training configuration file
%cd /content/gdrive/MyDrive/content_2/models/research/deploy
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

In [ ]:
pipeline_fname = '/content/gdrive/MyDrive/content_2/models/research/deploy/' + base_pipeline_file
fine_tune_checkpoint = '/content/gdrive/MyDrive/content_2/models/research/deploy/' + model_name + '/checkpoint/ckpt-0'
 
def get_num_classes(pbtxt_fname):
   from object_detection.utils import label_map_util
   label_map = label_map_util.load_labelmap('/content/gdrive/MyDrive/content_2/test/teams_label_map.pbtxt')
   categories = label_map_util.convert_label_map_to_categories(
       label_map, max_num_classes=90, use_display_name=True)
   category_index = label_map_util.create_category_index(categories)
   return len(category_index.keys())
num_classes = get_num_classes('/content/gdrive/MyDrive/content_2/test/teams_label_map.pbtxt')

In [ ]:
num_classes

In [ ]:
import re
%cd /content/gdrive/MyDrive/content_2/models/research/deploy
print('writing custom configuration file')
with open(pipeline_fname) as f:
   s = f.read()
with open('pipeline_file.config', 'w') as f:
   # fine_tune_checkpoint
   s = re.sub('fine_tune_checkpoint: ".*?"',
              'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
     # tfrecord files train and test.
   s = re.sub(
       '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format('/content/gdrive/MyDrive/content_2/train/teams.tfrecord'), s)
   s = re.sub(
       '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format('/content/gdrive/MyDrive/content_2/test/teams.tfrecord'), s)
   # label_map_path
   s = re.sub(
       'label_map_path: ".*?"', 'label_map_path: "{}"'.format('/content/gdrive/MyDrive/content_2/train/teams_label_map.pbtxt'), s)
   # Set training batch_size.
   s = re.sub('batch_size: [0-9]+',
              'batch_size: {}'.format(batch_size), s)
 
   # Set training steps, num_steps
   s = re.sub('num_steps: [0-9]+',
              'num_steps: {}'.format(num_steps), s)
   # Set number of classes num_classes.
   s = re.sub('num_classes: [0-9]+',
              'num_classes: {}'.format(num_classes), s)
     #fine-tune checkpoint type
   s = re.sub(
       'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
      
   f.write(s)

In [ ]:
#Show model configuration file
%cat /content/gdrive/MyDrive/content_2/models/research/deploy/pipeline_file.config

In [ ]:
#Specify directories
pipeline_file = '/content/gdrive/MyDrive/content_2/models/research/deploy/pipeline_file.config'
model_dir = '/content/gdrive/MyDrive/content_2/log_files'

In [ ]:
#Train the model
!python /content/gdrive/MyDrive/content_2/models/research/object_detection/model_main_tf2.py \
   --pipeline_config_path={pipeline_file} \
   --model_dir={model_dir} \
   --alsologtostderr \
   --num_train_steps={num_steps} \
   --sample_1_of_n_eval_examples=1 \
   --num_eval_steps={num_eval_steps}

In [ ]:
#Spicify directory where evaluation files will be saved
testing_dir = '/content/gdrive/MyDrive/content_2/log_files/testing'

In [ ]:
#model evaluation
!python /content/gdrive/MyDrive/content_2/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={testing_dir} \
    --checkpoint_dir={model_dir} 

In [ ]:
#See list of log files
%ls '/content/gdrive/MyDrive/content_2/log_files'

In [ ]:
#run conversion script
import re
import numpy as np
output_directory = '/content/gdrive/MyDrive/content_2/fine_tuned_model'
#place the model weights you would like to export here
last_model_path = '/content/gdrive/MyDrive/content_2/log_files/'
print(last_model_path)
!python /content/gdrive/MyDrive/content_2/models/research/object_detection/exporter_main_v2.py \
   --trained_checkpoint_dir {last_model_path} \
   --output_directory {output_directory} \
   --pipeline_config_path {pipeline_file}

In [ ]:
#recover the saved model
pipeline_config = pipeline_file
#generally you want to put the last ckpt from training in here
model_dir = '/content/gdrive/MyDrive/content_2/log_files/ckpt-51'
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
     model_config=model_config, is_training=False)
 
# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
     model=detection_model)
ckpt.restore(os.path.join('/content/gdrive/MyDrive/content_2/log_files/ckpt-51'))
 
 
def get_model_detection_function(model):
 """Get a tf.function for detection."""
 
 @tf.function
 def detect_fn(image):
   """Detect objects in image."""
 
   image, shapes = model.preprocess(image)
   prediction_dict = model.predict(image, shapes)
   detections = model.postprocess(prediction_dict, shapes)
 
   return detections, prediction_dict, tf.reshape(shapes, [-1])
 
 return detect_fn
 
detect_fn = get_model_detection_function(detection_model)

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
#without random
#map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
   label_map,
   max_num_classes=label_map_util.get_max_label_map_index(label_map),
   use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [ ]:
#Directory to save output images with bounding boxes
cd /content/gdrive/MyDrive/barca_bayern_box_images

In [ ]:
#Create bounding boxes on test images
from PIL import Image
from natsort import natsorted
TEST_IMAGE_PATHS = natsorted(glob.glob('/content/gdrive/MyDrive/barca_bayern/*.jpg'))
k=1
for i in range(len(TEST_IMAGE_PATHS)):
  image_path = TEST_IMAGE_PATHS[i]
  test_image = Image.open(image_path)
  image_np = load_image_into_numpy_array(test_image)

  input_tensor = tf.convert_to_tensor(
   np.expand_dims(image_np, 0), dtype=tf.float32)
  detections, predictions_dict, shapes = detect_fn(input_tensor)

  label_id_offset = 1
  image_np_with_detections = image_np.copy()
  
  viz_utils.visualize_boxes_and_labels_on_image_array(
     image_np_with_detections,
     detections['detection_boxes'][0].numpy(),
     (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
     detections['detection_scores'][0].numpy(),
     category_index,
     use_normalized_coordinates=True,
     max_boxes_to_draw=200,
     min_score_thresh=.5,
     agnostic_mode=False,
     )

  im = Image.fromarray(image_np_with_detections)
  im.save("{}.jpeg".format(k))


  plt.figure(figsize=(12,16))
  plt.imshow(image_np_with_detections)
  plt.show()

  k += 1
  print(k)
  
  
  


In [ ]:
#Convert Images to video
import os
from natsort import natsorted
import moviepy.video.io.ImageSequenceClip
image_folder='/content/gdrive/MyDrive/barca_bayern_box_images_2'
fps=25

image_files = [os.path.join(image_folder,img)
               for img in natsorted(os.listdir(image_folder))
               if img.endswith(".jpeg")]
clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
clip.write_videofile('my_video.mp4')